In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
from pymc.math import eq, switch

%load_ext watermark
%load_ext lab_black

# Equivalence of Generic and Brand-Name Drugs 

This is the first Bayesian ANOVA example. We're comparing the means of two different drug types.

Adapted from [Codes for Unit 6: equivalence.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit6/equivalence.odc).

## Associated lecture video: Unit 6 Lesson 7

In [7]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=58" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Problem Statement

The manufacturer wishes to demonstrate that their generic drug for a particular metabolic disorder is equivalent to a brand name drug. One of indication of the disorder is an abnormally low concentration of levocarnitine, an amino acid derivative, in the plasma. The treatment with the brand name drug substantially increases this concentration.

A small clinical trial is conducted with 43 patients, 18 in the Brand Name Drug arm and 25 in the Generic Drug arm. The increases in the log-concentration of levocarnitine are in the data below.

The FDA declares that bioequivalence among the two drugs can be established if the difference in response to the two drugs is within 2 units of log-concentration. Assuming that the log-concentration measurements follow normal distributions with equal population variance, can these two drugs be declared bioequivalent within a tolerance +/-2  units?

---
The way the data is set up in the .odc file is strange. It seems simpler to just have a separate list for each increase type.

In [2]:
# fmt: off
increase_type1 = [7, 8, 4, 6, 10, 10, 5, 7, 9, 8, 6, 7, 8, 4, 6, 10, 8, 9]
increase_type2 = [6, 7, 5, 9, 5, 5, 3, 7, 5, 10, 8, 5, 8, 4, 4, 8, 6, 11, 
                  7, 5, 5, 5, 7, 4, 6]
# fmt: on

We're using ```pm.math.switch``` and ```pm.math.eq``` to recreate the BUGS ```step()``` function for the ```probint``` variable.

See [Unit 6: Stress, Diet and Plasma Acids](https://areding.github.io/6420-pymc/unit6/Unit6-stressacids.html) to find out more about converting the BUGS step function.

In [4]:
with pm.Model() as m:
    # priors
    mu1 = pm.Normal("mu1", mu=10, sigma=316)
    mu2 = pm.Normal("mu2", mu=10, sigma=316)
    mudiff = pm.Deterministic("mudiff", mu1 - mu2)
    prec = pm.Gamma("prec", alpha=0.001, beta=0.001)
    sigma = 1 / pm.math.sqrt(prec)

    probint = pm.Deterministic(
        "probint",
        switch(mudiff + 2 >= 0, 1, 0) * switch(eq(2 - mudiff, 0), 1, 0),
    )

    y_type1 = pm.Normal("y_type1", mu=mu1, sigma=sigma, observed=increase_type1)
    y_type2 = pm.Normal("y_type2", mu=mu2, sigma=sigma, observed=increase_type2)

    # start sampling
    trace = pm.sample(
        10000,
        chains=4,
        tune=500,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/pymc/aesaraf.py:996: UserWarning: The parameter 'updates' of aesara.function() expects an OrderedDict, got <class 'dict'>. Using a standard dictionary here results in non-deterministic behavior. You should use an OrderedDict if you are using Python 2.7 (collections.OrderedDict for older python), or use a list of (shared, update) pairs. Do not just convert your dictionary to this type before the call as the conversion will still be non-deterministic.
  aesara_function = aesara.function(
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu1, mu2, prec]


/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 500 tune and 10_000 draw iterations (2_000 + 40_000 draws total) took 12 seconds.


In [5]:
az.summary(trace, hdi_prob=0.95)

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/arviz/stats/diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu1,7.335,0.479,6.405,8.285,0.002,0.002,45553.0,29526.0,1.0
mu2,6.201,0.403,5.410,6.996,0.002,0.001,48853.0,28445.0,1.0
prec,0.263,0.059,0.153,0.378,0.000,0.000,44056.0,27074.0,1.0
mudiff,1.133,0.626,-0.107,2.332,0.003,0.002,46056.0,28917.0,1.0
probint,0.000,0.000,0.000,0.000,0.000,0.000,40000.0,40000.0,NaN


BUGS results:

|          | mean   | sd      | MC_error | val2.5pc | median | val97.5pc | start | sample |
|----------|--------|---------|----------|----------|--------|-----------|-------|--------|
| mu[1]    | 7.332  | 0.473   | 0.001469 | 6.399    | 7.332  | 8.264     | 1001  | 100000 |
| mu[2]    | 6.198  | 0.4006  | 0.001213 | 5.406    | 6.199  | 6.985     | 1001  | 100000 |
| mudiff   | 1.133  | 0.618   | 0.00196  | -0.07884 | 1.134  | 2.354     | 1001  | 100000 |
| prec     | 0.2626 | 0.05792 | 1.90E-04 | 0.1617   | 0.2584 | 0.3877    | 1001  | 100000 |
| probint  | 0.9209 | 0.2699  | 9.07E-04 | 0        | 1      | 1         | 1001  | 100000 |

In [6]:
%watermark --iversions -v

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.3.0

numpy     : 1.22.3
pymc      : 4.0.0b5
matplotlib: 3.5.2
arviz     : 0.12.1

